# СЛЕПО ПРЕБАРУВАЊЕ

## Објаснување - пребарување по широчина vs по должина

По широчина:
* треба поќе меморија
* бара ниво по ниво
* го наоѓа најкраткио пат, поспоро
* се користи коќе не' интересира најкраткото решение
    * т.е. коќе на излез ја сакаме цела постапка, а **крајната состојба веќе знајме шо треба да биди**
    * пр. движење коњ во најмалце чекори, движење плочки сложувалка во најмалце чекори, преминвање мост/река

По должина:
* бара помалце меморија
* бара подстебло по подстебло
    * разгранва едно стебло до крај, па после backtrack-нува
        * ако некое од тие стебла е бесконечно, ќе се заглави во бесконечен loop
* не секогаш го наоѓа најкраткио пат, ама е побрз
* се користи коќе не' интересира едно решение, а не оптималното
    * т.е. коќе **на излез неќиме постапка, туку крајна состојба**
    * пр. еден начин за поставување кралици, судоку
 

## Граф

In [17]:
class Graph():
    
    def __init__(self):
        self.dict = {}
    
    def add_node(self, node):
        self.dict[node] = []
        
    def add_edge(self, link):
        node1, node2 = link
        self.dict[node1].append(node2)
        self.dict[node2].append(node1)
        
    def give_nodes(self):
        return list(self.dict)
    
    def give_edges(self):
        links = []
        for node1 in self.dict:
            for node2 in self.dict[node1]:
                links.append((node1, node2))
        return links
    
    def give_neighbors(self, node):
        return self.dict[node]

In [18]:
g = Graph()

In [19]:
g.add_node('S')
g.add_node('A')
g.add_node('B')
g.add_node('C')
g.add_node('D')
g.add_node('E')
g.add_node('F')
g.add_node('G')
g.add_node('H')
g.give_nodes()

['S', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [20]:
g.add_edge(('S', 'A'))
g.add_edge(('S', 'C'))
g.add_edge(('S', 'G'))
g.add_edge(('A', 'B'))
g.add_edge(('C', 'D'))
g.add_edge(('C', 'E'))
g.add_edge(('C', 'F'))
g.add_edge(('G', 'F'))
g.add_edge(('G', 'H'))
g.add_edge(('E', 'H'))

In [21]:
g.give_edges()

[('S', 'A'),
 ('S', 'C'),
 ('S', 'G'),
 ('A', 'S'),
 ('A', 'B'),
 ('B', 'A'),
 ('C', 'S'),
 ('C', 'D'),
 ('C', 'E'),
 ('C', 'F'),
 ('D', 'C'),
 ('E', 'C'),
 ('E', 'H'),
 ('F', 'C'),
 ('F', 'G'),
 ('G', 'S'),
 ('G', 'F'),
 ('G', 'H'),
 ('H', 'G'),
 ('H', 'E')]

In [24]:
from queue import deque

In [29]:
def breadth_first_search(graph, initial_node, goal_node):
    queue = deque([[initial_node]])
    visited = {initial_node}
    
    if initial_node == goal_node:
        return
    
    while queue:
        list_to_expand = queue.popleft()
        node_to_expand = list_to_expand[-1]
        
        for neighbor in graph.give_neighbors(node_to_expand):
            if neighbor not in visited:
                if neighbor == goal_node:
                    return list_to_expand + [neighbor]
                queue.append(list_to_expand + [neighbor])
                visited.add(neighbor)

In [30]:
breadth_first_search(g, 'S', 'E')

['S', 'C', 'E']

## Претурање вода од 3 сада

In [58]:
CAPACITY = (5, 8, 10)

In [59]:
def transfer_water(CAPACITY, current_state, source_index, sink_index):
    list_state = list(current_state)
    
    if CAPACITY[sink_index] - current_state[sink_index] >= current_state[source_index]:
        list_state[sink_index] = current_state[sink_index] + current_state[source_index]
        list_state[source_index] = 0
        return tuple(list_state)
    
    list_state[sink_index] = CAPACITY[sink_index]
    list_state[source_index] = current_state[source_index] - (CAPACITY[sink_index] - current_state[sink_index])
    return tuple(list_state)
    
# (3, 2, 10)
# (5, 7, 10)

In [60]:
 transfer_water(CAPACITY, (5, 7, 10), 0, 1) # primer

(4, 8, 10)

In [61]:
def expand_state(current_state):
    states = []
    
    for index, element in enumerate(current_state):
        list_state = list(current_state)
        if element > 0:
            list_state[index] = 0
            states.append(tuple(list_state))
            
    for index, element in enumerate(current_state):
        list_state = list(current_state)
        if element < CAPACITY[index]:
            list_state[index] = CAPACITY[index]
            states.append(tuple(list_state))
            
    for index1, element1 in enumerate(current_state):
        for index2, element2 in enumerate(current_state):
            if index1 != index2:
                source_index = index1
                sink_index = index2
                if transfer_water(CAPACITY, current_state, source_index, sink_index) not in states:
                    states.append(transfer_water(CAPACITY, current_state, source_index, sink_index))
                
    return states

In [62]:
expand_state((5, 8, 10))

[(0, 8, 10), (5, 0, 10), (5, 8, 0), (5, 8, 10)]

In [63]:
from queue import deque

In [66]:
def breadth_first_search(initial_state, goal_state):
    
    if initial_state == goal_state:
        return initial_state
    
    queue = deque([[initial_state]])
    visited = {initial_state}
    
    while queue:
        list_to_expand = queue.popleft()
        state_to_expand = list_to_expand[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return list_to_expand + [next_state]
                queue.append(list_to_expand + [next_state])
                visited.add(next_state)
    

In [67]:
breadth_first_search((0, 0, 0), (1, 0, 0))

[(0, 0, 0),
 (0, 8, 0),
 (5, 3, 0),
 (0, 3, 0),
 (3, 0, 0),
 (3, 8, 0),
 (3, 0, 8),
 (1, 0, 10),
 (1, 0, 0)]

## 8 кралици на шаховска табла, ниедна не се напаѓа

In [144]:
N = 8

In [145]:
def is_valid(current_state, new_queen_row, new_queen_col):
    
    new_queen_main_diagonal = new_queen_row - new_queen_col # 3
    new_queen_counter_diagonal = new_queen_row + new_queen_col # 1
    
    current_state_without_none = current_state[0:(len(current_state) - current_state.count(None))]
    
    for queen_row, queen_col in enumerate(current_state_without_none): # queen_row e index na torkata, queen_col e element vo torkata # (0, 0), (1, 2)
        if queen_row + queen_col == new_queen_counter_diagonal:
            return False
        if queen_row - queen_col == new_queen_main_diagonal:
            return False
        if queen_col == new_queen_col:
            return False
    return True
    
# 0 1 2 3

# 1 0 0 0  0
# 0 0 1 0  1
# x N x x  2
# x x x x  3

In [146]:
is_valid((0, None, None, None), 1, 2)

True

In [147]:
def expand_state(current_state):
    new_queen_row = len(current_state) - current_state.count(None)
    states = []
    
    for new_queen_col in range(N):
        state_list = list(current_state)
        if is_valid(current_state, new_queen_row, new_queen_col):
            state_list[new_queen_row] = new_queen_col
            states.append(tuple(state_list))
            
    return states   

# 0 1 2 3

# 1 0 0 0  0
# 0 0 1 0  1
# x x x x  2
# x x x x  3

# (0, 2, None, None)

# new_queen_row == red na kralica
# new_queen_col == kol na kralica

In [148]:
expand_state((0, 1, None, None))

[(0, 1, 3, None),
 (0, 1, 4, None),
 (0, 1, 5, None),
 (0, 1, 6, None),
 (0, 1, 7, None)]

In [149]:
from collections import deque

In [150]:
def depth_first_search(initial_state):
    
    queue = deque([initial_state])
    visited = {initial_state}
    
    while queue:
        state_to_expand = queue.popleft()
        #state_to_expand = list_to_expand[-1]
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state.count(None) == 0:
                    return next_state
                queue.appendleft(next_state)
                visited.add(next_state)
                   
 # [ (None, None, None, None) ]

In [151]:
depth_first_search((None, None, None, None, None, None, None, None))

(7, 3, 0, 2, 5, 1, 6, 4)

In [152]:
'''
   0 1 2 3 4 5 6 7
0  _ _ _ _ _ _ _ x
1  _ _ _ x _ _ _ _
2  x _ _ _ _ _ _ _
3  _ _ x _ _ _ _ _ 
4  _ _ _ _ _ x _ _
5  _ x _ _ _ _ _ _
6  _ _ _ _ _ _ x _
7  _ _ _ _ x _ _ _
'''

'\n   0 1 2 3 4 5 6 7\n0  _ _ _ _ _ _ _ x\n1  _ _ _ x _ _ _ _\n2  x _ _ _ _ _ _ _\n3  _ _ x _ _ _ _ _ \n4  _ _ _ _ _ x _ _\n5  _ x _ _ _ _ _ _\n6  _ _ _ _ _ _ x _\n7  _ _ _ _ x _ _ _\n'

## Судоку

In [153]:
import math
import copy

In [154]:
def is_valid(current_state, number, number_row, number_col):
   
    if number in current_state[number_row]:
        return False
    
    for row in current_state:
        if row[number_col] == number:
            return False
        
    if number_row - number_col == 0: # ako brojot go pisime vo glavnata dijagonala
        main_diagonal_elements = []
        for i, row in enumerate(current_state):
            for j in range(9):
                if i - j == 0:
                    main_diagonal_elements.append(row[j])         
        if number in main_diagonal_elements:
            return False
        
    if number_row + number_col == 8:
        counter_diagonal_elements = []
        for i, row in enumerate(current_state):
            for j in range(9):
                if i + j == 8:
                    counter_diagonal_elements.append(row[j])
        if number in counter_diagonal_elements:
            return False
        
    square_list = [
        [[], [], []],
        [[], [], []],
        [[], [], []]
    ]    
    square_row = math.floor(number_row / 3)
    square_col = math.floor(number_col / 3)
    
    for i in range(9):
        for j in range(9):
            square_i = math.floor(i / 3)
            square_j = math.floor(j / 3)
            square_list[square_i][square_j].append(current_state[i][j])
      
    if number in square_list[square_row][square_col]:
        return False
    
    return True  
    

# 9 redici so 9 koloni => lista od 9 torki, so po 9 elementi po torka
# [ (1, 3, None, None, 5, 6, None, None, None), (2, 5, ...) ]


In [177]:
state = (
    (None, None, None, None, 5, 6, 7, 8, None),
    (1, None, None, None, None, None, 3, None, None),
    (2, 7, None, 3, None, None, None, None, None),
    (3, None, None, None, None, None, 9, None, None),
    (4, None, None, None, None, None, None, None, 2),
    (None, None, 9, None, None, None, None, None, 3),
    (None, None, None, None, None, 7, None, 5, 4),
    (None, None, 2, None, None, None, None, None, 8),
    (None, 4, 5, 6, 3, None, None, None, None)
)

In [178]:
is_valid(state, 1, 0, 3) # primer

True

In [179]:
current_state[0][4]

5

In [180]:
def expand_state(current_state):
    states = []
    
    for i in range(9):
        do_break = False
        for j in range(9):
            if current_state[i][j] == None:
                number_row = i
                number_col = j
                do_break = True
                break
        if do_break:
            break
            
    for number in range(1, 10):
        if is_valid(current_state, number, number_row, number_col):
            new_state = []
            for row in current_state:
                new_state.append(list(row))
            
            new_state[number_row][number_col] = number
            new_state_tuple = []
            for row in new_state:
                new_state_tuple.append(tuple(row))
            states.append(tuple(new_state_tuple))
           
    return states

In [181]:
expand_state(state)

[((9, None, None, None, 5, 6, 7, 8, None),
  (1, None, None, None, None, None, 3, None, None),
  (2, 7, None, 3, None, None, None, None, None),
  (3, None, None, None, None, None, 9, None, None),
  (4, None, None, None, None, None, None, None, 2),
  (None, None, 9, None, None, None, None, None, 3),
  (None, None, None, None, None, 7, None, 5, 4),
  (None, None, 2, None, None, None, None, None, 8),
  (None, 4, 5, 6, 3, None, None, None, None))]

In [182]:
from collections import deque

In [2]:
def depth_first_search(initial_state):
    
    queue = deque([initial_state])
    visited = {initial_state}
    
    while queue:
        state_to_expand = queue.popleft()
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                how_many_nones = 0
                for row in next_state:
                    how_many_nones += row.count(None)
                if how_many_nones == 0:
                    return next_state
                queue.appendleft(next_state)
                visited.add(next_state)
                        
    

In [3]:
depth_first_search(state)

NameError: name 'state' is not defined

## Игра со плочки

In [3]:
# [ (0, 5, 3),
#   (7, 8, 4),
#   (2, 6, 1) ]

# (0, 1, 2, 3, 4, 5, 6, 7, 8)

state = ((0, 5, 3),
         (7, 8, 4),
         (2, 6, 1))

goal_state = ((0, 1, 2),
              (3, 4, 5),
              (6, 7, 8))

In [23]:
def move_tile(state, tile_position, zero_position):
    tile_row, tile_col = tile_position
    zero_row, zero_col = zero_position
    state_list = []
    for row in state:
        state_list.append(list(row))
        
    state_list[zero_row][zero_col] = state[tile_row][tile_col]
    state_list[tile_row][tile_col] = state[zero_row][zero_col]
    
    state_tuple = []
    for row in state_list:
        state_tuple.append(tuple(row))
        
    return tuple(state_tuple)
    

In [24]:
move_tile(state, (0, 1), (0, 0)) # primer

((5, 0, 3), (7, 8, 4), (2, 6, 1))

In [29]:
def expand_state(state):
    states_list = []
    
    for index_row, row in enumerate(state):
        if 0 in row:
            zero_position = (index_row, row.index(0))
            break
            
    zero_row, zero_col = zero_position
            
    tile_indexes_to_move = [(zero_row+1, zero_col),
                            (zero_row-1, zero_col),
                            (zero_row, zero_col+1),
                            (zero_row, zero_col-1)] 
    
            # [(1, 0), (0, 1)]
            
    for index_row, index_col in tile_indexes_to_move:
        if (index_row < 0 or index_row >= 3) or (index_col < 0 or index_col >= 3):
            continue
        states_list.append(move_tile(state, (index_row, index_col), zero_position))
        
    return states_list
            
    

In [30]:
expand_state(state)

[((7, 5, 3), (0, 8, 4), (2, 6, 1)), ((5, 0, 3), (7, 8, 4), (2, 6, 1))]

In [31]:
from collections import deque

def breadth_first_search(initial_state, goal_state):
    if initial_state == goal_state:
        return
    
    queue = deque([[initial_state]])
    visited = {initial_state}
    
    while queue:
        list_to_expand = queue.popleft()
        state_to_expand = list_to_expand[-1]
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return list_to_expand + [next_state]
                queue.append(list_to_expand + [next_state])
                visited.add(next_state)
                

In [34]:
%%time

breadth_first_search(state, goal_state)

CPU times: total: 3.97 s
Wall time: 4.05 s


[((0, 5, 3), (7, 8, 4), (2, 6, 1)),
 ((5, 0, 3), (7, 8, 4), (2, 6, 1)),
 ((5, 3, 0), (7, 8, 4), (2, 6, 1)),
 ((5, 3, 4), (7, 8, 0), (2, 6, 1)),
 ((5, 3, 4), (7, 0, 8), (2, 6, 1)),
 ((5, 3, 4), (7, 6, 8), (2, 0, 1)),
 ((5, 3, 4), (7, 6, 8), (0, 2, 1)),
 ((5, 3, 4), (0, 6, 8), (7, 2, 1)),
 ((5, 3, 4), (6, 0, 8), (7, 2, 1)),
 ((5, 3, 4), (6, 2, 8), (7, 0, 1)),
 ((5, 3, 4), (6, 2, 8), (7, 1, 0)),
 ((5, 3, 4), (6, 2, 0), (7, 1, 8)),
 ((5, 3, 4), (6, 0, 2), (7, 1, 8)),
 ((5, 3, 4), (6, 1, 2), (7, 0, 8)),
 ((5, 3, 4), (6, 1, 2), (0, 7, 8)),
 ((5, 3, 4), (0, 1, 2), (6, 7, 8)),
 ((0, 3, 4), (5, 1, 2), (6, 7, 8)),
 ((3, 0, 4), (5, 1, 2), (6, 7, 8)),
 ((3, 1, 4), (5, 0, 2), (6, 7, 8)),
 ((3, 1, 4), (0, 5, 2), (6, 7, 8)),
 ((0, 1, 4), (3, 5, 2), (6, 7, 8)),
 ((1, 0, 4), (3, 5, 2), (6, 7, 8)),
 ((1, 4, 0), (3, 5, 2), (6, 7, 8)),
 ((1, 4, 2), (3, 5, 0), (6, 7, 8)),
 ((1, 4, 2), (3, 0, 5), (6, 7, 8)),
 ((1, 0, 2), (3, 4, 5), (6, 7, 8)),
 ((0, 1, 2), (3, 4, 5), (6, 7, 8))]

## Преминување река - мажи и деца

Два мажи и две деца треба да ја преминат реката. 
   * Чамецот може да издржи само еден маж или пак две деца наеднаш.
   * Сите знаат да веслаат.

In [46]:
def one_person_goes(state, person_index):
    state_list = list(state)
    state_list[person_index] = state_list[4] = 1
    return tuple(state_list)

In [52]:
def two_kids_go(state):
    state_list = list(state)
    state_list[2] = state_list[3] = state_list[4] = 1
    return tuple(state_list)

In [48]:
def one_person_comes(state, person_index):
    state_list = list(state)
    state_list[person_index] = state_list[4] = 0
    return tuple(state_list)

In [53]:
def two_kids_come(state):
    state_list = list(state)
    state_list[2] = state_list[3] = state_list[4] = 0
    return tuple(state_list)

In [54]:
# (0, 0, 0, 0, 0) = (maz, maz, dete, dete, camec)

def expand_state(state):
    states = []
    
    if state[4] == 0: # nekoj treba da odi
        for index in range(4):
            if state[index] == 0:
                states.append(one_person_goes(state, index))
        
        if state[2] == 0 and state[3] == 0:
            states.append(two_kids_go(state))
            
    if state[4] == 1:
        for index in range(4):
            if state[index] == 1:
                states.append(one_person_comes(state, index))
                
        if state[2] == 1 and state[3] == 1:
            states.append(two_kids_come(state))
            
    return states
        

In [55]:
expand_state((0, 0, 0, 0, 0))

[(1, 0, 0, 0, 1),
 (0, 1, 0, 0, 1),
 (0, 0, 1, 0, 1),
 (0, 0, 0, 1, 1),
 (0, 0, 1, 1, 1)]

In [58]:
from collections import deque

def breadth_first_search(initial_state, goal_state):
    if initial_state == goal_state:
        return
    
    queue = deque([[initial_state]])
    visited = {initial_state}
    
    while queue:
        list_to_expand = queue.popleft()
        state_to_expand = list_to_expand[-1]
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return list_to_expand + [next_state]
                queue.append(list_to_expand + [next_state])
                visited.add(next_state)
                

In [59]:
breadth_first_search((0, 0, 0, 0, 0), (1, 1, 1, 1, 1))

[(0, 0, 0, 0, 0),
 (0, 0, 1, 1, 1),
 (0, 0, 0, 1, 0),
 (1, 0, 0, 1, 1),
 (1, 0, 0, 0, 0),
 (1, 0, 1, 1, 1),
 (1, 0, 0, 1, 0),
 (1, 1, 0, 1, 1),
 (1, 1, 0, 0, 0),
 (1, 1, 1, 1, 1)]

## Преминување река - фамилија

Шест луѓе треба да ја преминат реката. Дедо, баба, татко, мајка, момче и девојче.
   * Само мажите знаат да веслаат.
   * Чамецот може да издржи највеќе двајца.
   * Дедото не сака да биде придружуван во чамецот.
   * Девојчето може да ја премине реката единствено со татко и.

In [96]:
# (grandma, grandpa, dad, mom, boy, girl, boat)

grandma_index, grandpa_index, dad_index, mom_index, boy_index, girl_index, boat_index = range(7)

In [97]:
def one_person_goes(state, person_index):
    state_list = list(state)
    state_list[person_index] = state_list[boat_index] = 1
    #print(state_list)
    return tuple(state_list)

In [98]:
one_person_goes((0, 0, 0, 0, 0, 0, 0), 0)

(1, 0, 0, 0, 0, 0, 1)

In [99]:
def two_people_go(state, person1_index, person2_index):
    state_list = list(state)
    state_list[person1_index] = state_list[person2_index] = state_list[boat_index] = 1
    return tuple(state_list)

In [100]:
two_people_go((0, 0, 0, 0, 0, 0, 0), 0, 1)

(1, 1, 0, 0, 0, 0, 1)

In [101]:
def one_person_comes(state, person_index):
    state_list = list(state)
    state_list[person_index] = state_list[boat_index] = 0
    return tuple(state_list)

In [102]:
def two_people_come(state, person1_index, person2_index):
    state_list = list(state)
    state_list[person1_index] = state_list[person2_index] = state_list[boat_index] = 0
    return tuple(state_list)

In [103]:
def expand_state(state):
    
    states = []
    
    if state[boat_index] == 0:
        
        if state[grandpa_index] == 0:
            states.append(one_person_goes(state, grandpa_index))
        
        if state[dad_index] == 0 and state[girl_index] == 0:
            states.append(two_people_go(state, dad_index, girl_index))
            
        for man_index in [dad_index, boy_index]:
            if state[man_index] == 0:
                states.append(one_person_goes(state, man_index))

                for other_index in [grandma_index, dad_index, boy_index, mom_index]:
                    if man_index != other_index and state[other_index] == 0:
                        states.append(two_people_go(state, man_index, other_index))
                        
    if state[boat_index] == 1:
        
        if state[grandpa_index] == 1:
            states.append(one_person_comes(state, grandpa_index))
            
        if state[dad_index] == 1 and state[girl_index] == 1:
            states.append(two_people_come(state, dad_index, girl_index))
            
        for man_index in [dad_index, boy_index]:
            if state[man_index] == 1:
                states.append(one_person_comes(state, man_index))
                
                for other_index in [grandma_index, dad_index, boy_index, mom_index]:
                    if man_index != other_index and state[other_index] == 1:
                        states.append(two_people_come(state, man_index, other_index))
                        
    return states

In [104]:
expand_state((0, 0, 0, 0, 0, 0, 0))
# (grandma, grandpa, dad, mom, boy, girl, boat)

[(0, 1, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 1, 1),
 (0, 0, 1, 0, 0, 0, 1),
 (1, 0, 1, 0, 0, 0, 1),
 (0, 0, 1, 0, 1, 0, 1),
 (0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 1),
 (1, 0, 0, 0, 1, 0, 1),
 (0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 1, 1, 0, 1)]

In [109]:
from collections import deque

def breadth_first_search(initial_state, goal_state):
    
    if initial_state == goal_state:
        return
    
    queue = deque([[initial_state]])
    visited = {initial_state}
    
    while queue:
        list_to_expand = queue.popleft()
        state_to_expand = list_to_expand[-1]
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return list_to_expand + [next_state]
                visited.add(next_state)
                queue.append(list_to_expand + [next_state])
                
    
    

In [111]:
breadth_first_search((0,)*7, (1,)*7)
# (grandma, grandpa, dad, mom, boy, girl, boat)

[(0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 0),
 (1, 0, 1, 0, 0, 1, 1),
 (1, 0, 0, 0, 0, 1, 0),
 (1, 0, 1, 0, 1, 1, 1),
 (1, 0, 0, 0, 1, 1, 0),
 (1, 1, 0, 0, 1, 1, 1),
 (1, 1, 0, 0, 0, 1, 0),
 (1, 1, 1, 0, 1, 1, 1),
 (1, 1, 0, 0, 1, 1, 0),
 (1, 1, 1, 1, 1, 1, 1)]

## Движење на коњ по шаховска табла

In [2]:
BOARD_SIZE = 8

In [3]:
def is_valid(movement):
    if 0 <= movement[0] < BOARD_SIZE and 0 <= movement[1] < BOARD_SIZE:
        return True
    return False

In [8]:
is_valid((-1, 1))

False

In [10]:
def expand_state(state):
    # state == pozicijata na konjot vo momentot, torka
    x, y = state
    next_movements = [(x+1, y+2), (x+1, y-2), (x-1, y+2), (x-1, y-2),
                      (x+2, y-1), (x+2, y+1), (x-2, y-1), (x-2, y+1)]
    states = []
    for movement in next_movements:
        if is_valid(movement):
            states.append(movement)
    return states

In [22]:
expand_state((0, 4))

[(1, 6), (1, 2), (2, 3), (2, 5)]

In [27]:
from collections import deque

def breadth_first_search(initial_state, goal_row):
    
    queue = deque([[initial_state]])
    visited = {initial_state}
    
    while queue:
        
        list_to_expand = queue.popleft()
        state_to_expand = list_to_expand[-1]
        
        for next_state in expand_state(state_to_expand):
            if next_state[0] == goal_row:
                return list_to_expand + [next_state]
            if next_state not in visited:
                queue.append(list_to_expand + [next_state])
                visited.add(next_state)

In [26]:
breadth_first_search((0, 3), 7)

[(0, 3), (1, 5), (3, 4), (5, 3), (7, 2)]

## Движење на коњ по шаховска табла - детално објаснато

In [2]:
from collections import deque

In [3]:
def is_valid(position):
    # is_valid е функција која проверува дали позицијата на коњот е валидна,
    # (позицијата е невалидна кога коњот се наоѓа надвор од шаховската табла).
    # Aргументот кој се испраќа во функцијата е торка. Taa соодветствува на
    # редот и колоната каде се наоѓа коњот.
    
    position_row, position_column = position
    if position_row < 0 or position_row >= 8:
        return False 
    if position_column < 0 or position_column >= 8:
        return False
    return True

In [4]:
is_valid((-3, 2)) # пример

False

In [5]:
is_valid((3, 4)) # пример

True

In [6]:
def expand_state(position_row, position_column):
    movements = [] # листа од сите движења кои смее да ги направи коњот, кога тој ќе се наоѓа во дадена позиција
        
    additions = [(2,1), (2, -1), (-2,1), (-2,-1), (-1,2), (-1,-2), (1, 2), (1, -2)]
        # Коњот се движи во Г форма, односно: или оди 2 полиња вертикално (нагоре (+) и надолу(-)),  а 1
        # поле хоризонтално (налево (-) и надесно(+)), или обратно: 1 поле вертикално, а 2 полиња хоризонтално.
        
        # Елементите од листата additions се торки чиј прв елемент ќе се додаде на 
        # моменталниот ред каде се наоѓа коњот, а вториот елемент од торката ќе се додаде
        # на моменталната колона каде се наоѓа коњот. Вака ќе симулираме придвижување на коњот
        # по шаховската табла.
        
    for i, j in additions:
        position = (position_row + i, position_column + j)
        if is_valid(position):
            movements.append(position)
            
    return movements

In [7]:
expand_state(0,4) # пример

[(2, 5), (2, 3), (1, 6), (1, 2)]

In [8]:
def search(initial_position):
    if not is_valid(initial_position):
        print("Првобитната позиција на коњот е невалидна.")
        return
    if initial_position[0] == 7:
        print("Веќе сте на крајот од таблата.")
        return
    
    visited = {initial_position}
    positions_queue = deque([[initial_position]]) # positions_queue е листа од листи. Секоја
        # внатрешна листа претставува патека на движење т.е. е листа од торки - секоја наредна
        # торка во една листа ја прикажува наредната позиција на која што може да оди коњот. 
        
        # Користиме слепо пребарување прво по широчина, бидејќи така ќе се најде најкратката патека 
        # до крајот на шаховската табла (крајот на таблата е редица 7 во овој случај).
    
    while positions_queue:
        list_to_expand = positions_queue.popleft()
        position_to_expand = list_to_expand[-1]
        position_row, position_column = position_to_expand
        
        for position in expand_state(position_row, position_column):
            if position not in visited:
                position_row, position_column = position
                if position_row == 7:
                    return list_to_expand + [position]
                visited.add(position)
                positions_queue.append(list_to_expand + [position])

In [9]:
search((0, 11)) # пример

Првобитната позиција на коњот е невалидна.


In [10]:
search((7, 0)) # пример

Веќе сте на крајот од таблата.


In [11]:
search((0, 3)) # пример

[(0, 3), (2, 4), (4, 5), (6, 6), (7, 4)]